In [225]:
%load_ext autoreload
%autoreload 2
import os
import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import copy 

import textwrap

import warnings
warnings.filterwarnings('ignore') # suppress sklearn deprecation warnings for now..
try:
    import graphviz
except ModuleNotFoundError as e:
    import pip
    pip.main(["install","graphviz"])
    import graphviz

from typing import Union

root_path = root_path = os.path.realpath('../../../..')
# print(root_path)
try:
    import causaltune
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "causaltune"))

from sklearn.model_selection import train_test_split
from causaltune import CausalTune
from causaltune.data_utils import CausalityDataset
from causaltune.datasets import generate_synthetic_data

# Import linear synthetic data creation
from causaltune.datasets import generate_linear_synthetic_data
from causaltune.models.passthrough import passthrough_model
from causaltune.datasets import load_dataset, save_dataset





# set a few params
metrics = [
    # "frobenius_norm",
     "psw_frobenius_norm",
    # "prob_erupt",
    #"policy_risk",
    # "codec",
    # "energy_distance", 
    # "psw_energy_distance",
    # "bite"
    ]

iv_metrics = [
    # "energy_distance",
    # "codec", 
    "frobenius_norm",                      
    ]

estimator_list = [
            "Dummy",
            "SparseLinearDML",
            "ForestDRLearner",
            "TransformedOutcome",
            "CausalForestDML",
            ".LinearDML",
            "DomainAdaptationLearner",
            "SLearner",
            "XLearner",
            "TLearner",
            #"Ortho"        
             ] 

iv_estimator_list = [
                 'iv.econml.iv.dr.LinearDRIV', 
                 #'iv.econml.iv.dml.OrthoIV', 
                 'iv.econml.iv.dml.DMLIV',
                 'iv.econml.iv.dr.SparseLinearDRIV',
                 'iv.econml.iv.dr.LinearIntentToTreatDRIV'
                 ] 


# More Parameters
n_runs = 1
num_samples = -1

test_size = 0.33 # equal train,val,test

time_budget = None
components_time_budget = 10

propensity_model='dummy'

filename_out = ""
out_dir = "PSW_FROBENIUS_TESTS"

if not os.path.exists(out_dir):
    os.makedirs(out_dir)





# Initialize an empty dictionary to store the loaded datasets
data_sets = {}

# List of dataset names and file paths
dataset_names = ['Linear_RCT']#, 'NonLinear_KC', 'NonLinear_KCKP', 'NonLinear_IV', 
                 #'Linear_RCT', 'Linear_KC', 'Linear_KCKP', 'Linear_IV']
file_paths = [f"RunDatasets/Small/{name}.pkl" for name in dataset_names]

# Loop through dataset names and file paths to load each dataset
for name, file_path in zip(dataset_names, file_paths):
    data_sets[name] = load_dataset(file_path)

# Optionally, print the keys of the dictionary to verify successful loading
print(f"Loaded datasets: {list(data_sets.keys())}")



for dataset_name, cd in data_sets.items():
    
    for i_run in range(1,n_runs+1):
        
        cd_i = copy.deepcopy(cd)
        train_df, test_df = train_test_split(cd_i.data, test_size=test_size)
        test_df = test_df.reset_index(drop=True)
        cd_i.data = train_df
        
        #for metric in metrics:
        for metric in metrics:
            
            #propensity_model=passthrough_model(
            #    cd.propensity_modifiers, include_control=False
            #)
            ct = CausalTune(
                metric=metric,
                estimator_list=estimator_list,

                num_samples = num_samples,
                time_budget=time_budget,
                components_time_budget=components_time_budget,               
                
                metrics_to_report=metrics,
                verbose=1,
                components_verbose=1,
                store_all_estimators=True,

                #propensity_model=propensity_model,
            )
            
            ct.fit(
                data=cd_i,
                treatment="treatment",
                outcome="outcome",
            )

            
            # compute relevant scores (skip newdummy)
            datasets = {"train": ct.train_df, "validation": ct.test_df, "test": test_df}
            # get scores on train,val,test for each trial, 
            # sort trials by validation set performance
            # assign trials to estimators
            estimator_scores = {est: [] for est in ct.scores.keys() if "NewDummy" not in est}
            for trial in ct.results.trials:
                # estimator name:
                estimator_name = trial.last_result["estimator_name"]
                if  trial.last_result["estimator"]:
                    estimator = trial.last_result["estimator"]
                    scores = {}
                    for ds_name, df in datasets.items():
                        scores[ds_name] = {}
                        # make scores
                        est_scores = ct.scorer.make_scores(
                            estimator,
                            df,
                            metrics_to_report=ct.metrics_to_report,
                        )
                        
                        # add cate:
                        scores[ds_name]["CATE_estimate"] = estimator.estimator.effect(df)
                        # add ground truth for convenience
                        scores[ds_name]["CATE_groundtruth"] = df["true_effect"]
                        scores[ds_name][metric] = est_scores[metric]
                    scores['optimization_score'] = trial.last_result.get('optimization_score')

                    estimator_scores[estimator_name].append(scores)


            # sort trials by validation performance
            for k in estimator_scores.keys():
                estimator_scores[k] = sorted(
                    estimator_scores[k],
                    key=lambda x: x["validation"][metric],
                    reverse=False if metric in ["energy_distance", 
                                                "psw_energy_distance", 
                                                "codec", 
                                                "frobenius_norm", 
                                                "psw_frobenius_norm",
                                                "policy_risk"] else True,
                )
            results = {
                "best_estimator": ct.best_estimator,
                "best_config": ct.best_config,
                "best_score": ct.best_score,
                "optimised_metric": metric,
                "scores_per_estimator": estimator_scores,
            }


            with open(f"{out_dir}/{filename_out}_{metric}_run_{i_run}_{dataset_name}.pkl", "wb") as f:
                pickle.dump(results, f)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
